#### Let's preprocess each column

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import random
from scipy.stats import truncnorm

### Reading the excel file

In [6327]:
df = pd.read_excel("../data/processed/preprocessed_data.xlsx")

In [6328]:
df.columns

Index(['id', 'acceptance_rate', 'institution', 'program', 'degree_type',
       'decision', 'undergrad_gpa', 'gre_quantitative_reasoning',
       'gre_verbal_reasoning', 'analytical_writing', 'notes', 'gre_total'],
      dtype='object')

### Checking the maximum and minimum acceptance rate range to see whether it is valid or not.

In [6329]:
df['acceptance_rate'] = pd.to_numeric(df['acceptance_rate'].str.replace('%', ''), errors='coerce')

In [6330]:
acceptance_rate_max = df['acceptance_rate'].max(skipna=True)
acceptance_rate_min = df['acceptance_rate'].min(skipna=True)

print("Max:", acceptance_rate_max)
print("Min:", acceptance_rate_min)


Max: 100.0
Min: 6.0


### Since the range is between 6% to 100%, the acceptance rate data seems fairly valid

_____________________________________________________________________________________________________________________________________________________________________________________

### Converting the gpa to numeric values

In [6331]:
df['undergrad_gpa'] = pd.to_numeric(df['undergrad_gpa'], errors='coerce')

### Checking the range of maximum and mininum of gpa

In [6332]:
undergrad_gpa_max = df['undergrad_gpa'].max(skipna=True)
undergrad_gpa_min = df['undergrad_gpa'].min(skipna=True)

print("Max:", undergrad_gpa_max)
print("Min:", undergrad_gpa_min)


Max: 332.0
Min: 0.01


### Valid GPA case 1

In [6333]:
gpa_greater_than_equal_to_one = df['undergrad_gpa'] >= 1.0
gpa_less_than_equal_to_four = df['undergrad_gpa'] <= 4.0
valid_gpa_between_one_and_four = df[gpa_greater_than_equal_to_one & gpa_less_than_equal_to_four]
len(valid_gpa_between_one_and_four)

44138

### Valid GPA Case 2
##### Sometimes the user may enter the gpa without the '.', hence we should check if someone has entered values between 100 to 332(since 332 is the vax value)

In [6334]:
gpa_greater_than_equal_to_hundred = df['undergrad_gpa'] >= 100
gpa_less_than_equal_to_four_hundred = df['undergrad_gpa'] < 401
valid_gpa_between_one_hundred_and_below_four_hundred = df.loc[(gpa_greater_than_equal_to_hundred) & (gpa_less_than_equal_to_four_hundred)]
valid_gpa_between_one_hundred_and_below_four_hundred

id  acceptance_rate               institution      program  \
23246  965397              NaN  University of California  Informatics   

      degree_type  decision  undergrad_gpa  gre_quantitative_reasoning  \
23246         PhD  Accepted          332.0                         NaN   

       gre_verbal_reasoning   analytical_writing       notes  gre_total  
23246                   NaN  interviewed on 1/28  Acceptance        NaN

In [6335]:
condition = gpa_greater_than_equal_to_hundred & gpa_less_than_equal_to_four_hundred

In [6336]:
df.loc[condition, 'undergrad_gpa'] = df.loc[condition, 'undergrad_gpa'] / 100

In [6337]:
fresh_mask = (df['undergrad_gpa'] >= 100) & (df['undergrad_gpa'] <= 400)
len(df.loc[fresh_mask])

0

### Invalid GPA

In [6338]:
# Step 1: Identify invalid GPA entries
invalid_gpa_mask = (
    (df['undergrad_gpa'] < 1.0) |
    (df['undergrad_gpa'] > 4.0) |  # You probably meant 4.0, not 400
    (df['undergrad_gpa'].isna())
)

In [6339]:
# Step 2: Generate random GPAs in a realistic range (around 3.8)
random_gpas = np.clip(
    np.random.normal(loc=3.8, scale=0.1, size=invalid_gpa_mask.sum()),
    3.6, 4.0
)

# Step 3: Replace invalid entries with these generated values
df.loc[invalid_gpa_mask, 'undergrad_gpa'] = random_gpas

In [6340]:
df['undergrad_gpa'].isnull().sum()

0

_____________________________________________________________________________________________________________________________________________________

### The GRE General Test has three scoring sections, each with its own score range:

#### Verbal Reasoning : Score range: 130 to 170

#### Quantitative Reasoning : Score range: 130 to 170

#### Analytical Writing : Score range: 0 to 6 

In [6341]:
# Convert GRE scores to numeric
list_of_gre_columns = ['gre_quantitative_reasoning', 'gre_verbal_reasoning', 'analytical_writing']
df[list_of_gre_columns[0]] = pd.to_numeric(df[list_of_gre_columns[0]], errors = 'coerce')
df[list_of_gre_columns[1]] = pd.to_numeric(df[list_of_gre_columns[1]], errors = 'coerce')

In [6342]:
gre_quantitative_reasoning_min = df[list_of_gre_columns[0]] > 129.0
gre_quantitative_reasoning_max = df[list_of_gre_columns[0]] < 171.0
valid_gre_quantitative_reasoning_profiles = df[gre_quantitative_reasoning_min & gre_quantitative_reasoning_max]
len(valid_gre_quantitative_reasoning_profiles)

10687

In [6343]:
gre_verbal_reasoning_min = df[list_of_gre_columns[1]] > 129.0
gre_verbal_reasoning_max = df[list_of_gre_columns[1]] < 171.0
valid_gre_verbal_reasoning_profiles = df[gre_verbal_reasoning_min & gre_verbal_reasoning_max]
len(valid_gre_verbal_reasoning_profiles)

11026

### Approach
Maybe a person entered their entire score instead of gre general. In that case the score boundary should be within 260 to 340. Else if it exceeds that then the same is neglected.

In [6344]:
invalid_gre_quantitative_reasoning_profiles = df[~(gre_quantitative_reasoning_min & gre_quantitative_reasoning_max) & ~df[list_of_gre_columns[0]].isna()]
len(invalid_gre_quantitative_reasoning_profiles)

1568

In [6345]:
invalid_gre_verbal_reasoning_profiles = df[~(gre_verbal_reasoning_min & gre_verbal_reasoning_max) & ~df[list_of_gre_columns[1]].isna()]
len(invalid_gre_verbal_reasoning_profiles)

68

#### Assigning values from 260 to 340 to gre_total column

In [6346]:
df['gre_total'] = df.loc[(df[list_of_gre_columns[0]] >= 260) & (df[list_of_gre_columns[0]] <= 340), list_of_gre_columns[0]]

In [6347]:
df['gre_total'] = df.loc[(df[list_of_gre_columns[1]] >= 260) & (df[list_of_gre_columns[1]] <= 340), list_of_gre_columns[1]]

#### Removing values 260 to 340 from gre_general column to gre_total column

In [6348]:
df.loc[(df[list_of_gre_columns[0]] >= 260) & (df[list_of_gre_columns[0]] <= 340), list_of_gre_columns[0]] = 0

In [6349]:
df.loc[(df[list_of_gre_columns[1]] >= 260) & (df[list_of_gre_columns[1]] <= 340), list_of_gre_columns[1]] = 0

### Re-checking if there are any values between 260 and 340

In [6350]:
len(df.loc[(df[list_of_gre_columns[0]] >= 260) & (df[list_of_gre_columns[0]] <= 340)])

0

In [6351]:
len(df.loc[(df[list_of_gre_columns[1]] >= 260) & (df[list_of_gre_columns[1]] <= 340)])

0

## Filling in empty values for GRE quantitative reasoning

### Histogram for GRE Quantitative reasoning

In [6352]:
median_val = df[list_of_gre_columns[0]].median()
mode_val = df[list_of_gre_columns[0]].mode()[0]  # mode() returns a Series
mean_val = df[list_of_gre_columns[0]].mean()

print(f"Median: {median_val}, Mode: {mode_val}, Mean: {mean_val}")


Median: 165.0, Mode: 170.0, Mean: 144.23215014279884


### Checking rows with values :
#### Below 130 
#### Above 340
#### Between 170 to 260

#### Checking for quantitative values

In [6353]:
df = pd.read_excel("../data/processed/preprocessed_final.xlsx")

In [6354]:
median_val = df[list_of_gre_columns[0]].median()
mode_val = df[list_of_gre_columns[0]].mode()[0]  # mode() returns a Series
mean_val = df[list_of_gre_columns[0]].mean()

print(f"Median: {median_val}, Mode: {mode_val}, Mean: {mean_val}")


Median: 155.0, Mode: 169, Mean: 152.4017192546584


In [6355]:
rows_above_340_gre_quantitative_reasoning = (
    (df[list_of_gre_columns[0]] > 340) |
    (df[list_of_gre_columns[0]] < 130) |
    ((df[list_of_gre_columns[0]] > 170) &
    (df[list_of_gre_columns[0]] < 340))  # This overlaps with < 130 and < 170
).sum()

print(f"Invalid rows for {list_of_gre_columns}: {rows_above_340_gre_quantitative_reasoning}")

Invalid rows for ['gre_quantitative_reasoning', 'gre_verbal_reasoning', 'analytical_writing']: 0


In [6356]:
df.loc[
    (df[list_of_gre_columns[0]] < 130) |
    (df[list_of_gre_columns[0]] > 170),
    list_of_gre_columns[0]
] = 0

In [6357]:
df.loc[(df[list_of_gre_columns[0]] > 340) | (df[list_of_gre_columns[0]] < 130) | ((df[list_of_gre_columns[0]] > 170) & (df[list_of_gre_columns[0]] < 340))]

Empty DataFrame
Columns: [Unnamed: 0.2, Unnamed: 0.1, Unnamed: 0, id, acceptance_rate, institution, program, degree_type, decision, undergrad_gpa, gre_quantitative_reasoning, gre_verbal_reasoning, analytical_writing, notes, gre_total, decision_encoded]
Index: []

In [6358]:
fig = px.histogram(valid_gre_quantitative_reasoning_profiles, x=list_of_gre_columns[0], nbins=30,
                   title=f'Distribution of Valid {list_of_gre_columns[0]} Scores')
fig.show()

In [6359]:
# Step 1: Get indices of rows where GRE score is missing or invalid
target_mask = (df['gre_quantitative_reasoning'].isna()) | (df['gre_quantitative_reasoning'] == 0)
target_indices = df[target_mask].index
missing_count = len(target_indices)  # Update count based on actual target rows

# Step 2–4: Generate tiered imputed values
low_pct = 0.40
mid_pct = 0.20
high_pct = 0.40

low_count = int(low_pct * missing_count)
mid_count = int(mid_pct * missing_count)
high_count = missing_count - low_count - mid_count  # ensure total matches

low_vals = np.random.uniform(130, 140, low_count)
mid_vals = np.random.uniform(140, 160, mid_count)
high_vals = np.random.uniform(165, 170, high_count)

combined_vals = np.concatenate([low_vals, mid_vals, high_vals])
rounded_vals = np.round(combined_vals)
rounded_vals[rounded_vals == 130] = 131

# Step 5: Assign back
df.loc[target_indices, 'gre_quantitative_reasoning'] = rounded_vals


In [6360]:
median_val = df[list_of_gre_columns[0]].median()
mode_val = df[list_of_gre_columns[0]].mode()[0]  # mode() returns a Series
mean_val = df[list_of_gre_columns[0]].mean()

print(f"Median: {median_val}, Mode: {mode_val}, Mean: {mean_val}")


Median: 155.0, Mode: 169, Mean: 152.4017192546584


In [6361]:
df['gre_quantitative_reasoning'].isna().sum()

0

### Checking verbal values

In [6362]:
median_val = df[list_of_gre_columns[1]].median()
mode_val = df[list_of_gre_columns[1]].mode()[0]  # mode() returns a Series
mean_val = df[list_of_gre_columns[1]].mean()

print(f"Median: {median_val}, Mode: {mode_val}, Mean: {mean_val}")


Median: 149.0, Mode: 148, Mean: 148.9225242236025


In [6363]:
df.loc[(df[list_of_gre_columns[1]] > 340) | (df[list_of_gre_columns[1]] < 130) | ((df[list_of_gre_columns[1]] > 170) & (df[list_of_gre_columns[0]] < 340))]

Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0      id  acceptance_rate  \
679              679           679         679  873277             33.0   
3357            3357          3357        3357  872415             45.0   
3628            3628          3628        3628  876294             23.0   
5955            5955          5955        5955  876603             24.0   
8164            8164          8164        8164  862452             38.0   
...              ...           ...         ...     ...              ...   
97589          97589         97589       97589  743681             44.0   
99329          99329         99329       99329  741337             45.0   
99344          99344         99344       99344  741315             39.0   
100027        100027        100027      100027  740314             40.0   
100028        100028        100028      100028  740313             36.0   

                              institution  \
679                       Rice University   
3357                Georgetown University   
3628                      Yale University   
5955                  Stanford University   
8164           Carnegie Mellon University   
...                                   ...   
97589              University Of Illinois   
99329   University of Southern California   
99344    Virginia Commonwealth University   
100027           West Virginia University   
100028            University of Rochester   

                                     program degree_type     decision  \
679                               Philosophy         PhD     Accepted   
3357                               Chemistry         PhD     Accepted   
3628                               Economics         PhD     Rejected   
5955                               Economics         PhD  Wait listed   
8164     Electrical And Computer Engineering     Masters     Accepted   
...                                      ...         ...          ...   
97589                         Bioengineering         PhD     Accepted   
99329   Mechanical And Aerospace Engineering     Masters     Accepted   
99344                     Genetic Counseling     Masters     Accepted   
100027                            Accounting         PhD     Rejected   
100028                            Accounting         PhD     Rejected   

        undergrad_gpa  gre_quantitative_reasoning  gre_verbal_reasoning  \
679              3.90                         169                     0   
3357             3.02                         140                     0   
3628             4.00                         139                     0   
5955             2.20                         138                     0   
8164             3.80                         136                     0   
...               ...                         ...                   ...   
97589            3.50                         165                     0   
99329            3.65                         161                     0   
99344            3.63                         165                     0   
100027           3.97                         169                     0   
100028           3.97                         169                     0   

        analytical_writing                                              notes  \
679                    3.8                              email to check portal   
3357                   2.0                                                NaN   
3628                   5.9  After what I did to Rachel, the dean of admiss...   
5955                   2.6  The groundhog saw its shadow, predicting 6 mor...   
8164                   3.5  Received the status update email around 12:30 ...   
...                    ...                                                ...   
97589                  5.3                                                NaN   
99329                  5.0                                                NaN   
99344                  4.0                     Applied to 6

In [6364]:
rows_above_340_gre_verbal_reasoning = (
    (df[list_of_gre_columns[1]] > 340) |
    (df[list_of_gre_columns[1]] < 130) |
    ((df[list_of_gre_columns[1]] > 170) &
    (df[list_of_gre_columns[1]] < 340))  # This overlaps with < 130 and < 170
).sum()

print(f"Invalid rows for {list_of_gre_columns}: {rows_above_340_gre_verbal_reasoning}")

Invalid rows for ['gre_quantitative_reasoning', 'gre_verbal_reasoning', 'analytical_writing']: 68


In [6365]:
rows_above_340_gre_verbal_reasoning = df.loc[
    (df[list_of_gre_columns[1]] < 130) |
    (df[list_of_gre_columns[1]] > 170),
    list_of_gre_columns[1]
] = 0


In [6366]:
df.loc[(df[list_of_gre_columns[1]] > 340) | (df[list_of_gre_columns[1]] < 130) | ((df[list_of_gre_columns[1]] > 170) & (df[list_of_gre_columns[1]] < 340))]

Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0      id  acceptance_rate  \
679              679           679         679  873277             33.0   
3357            3357          3357        3357  872415             45.0   
3628            3628          3628        3628  876294             23.0   
5955            5955          5955        5955  876603             24.0   
8164            8164          8164        8164  862452             38.0   
...              ...           ...         ...     ...              ...   
97589          97589         97589       97589  743681             44.0   
99329          99329         99329       99329  741337             45.0   
99344          99344         99344       99344  741315             39.0   
100027        100027        100027      100027  740314             40.0   
100028        100028        100028      100028  740313             36.0   

                              institution  \
679                       Rice University   
3357                Georgetown University   
3628                      Yale University   
5955                  Stanford University   
8164           Carnegie Mellon University   
...                                   ...   
97589              University Of Illinois   
99329   University of Southern California   
99344    Virginia Commonwealth University   
100027           West Virginia University   
100028            University of Rochester   

                                     program degree_type     decision  \
679                               Philosophy         PhD     Accepted   
3357                               Chemistry         PhD     Accepted   
3628                               Economics         PhD     Rejected   
5955                               Economics         PhD  Wait listed   
8164     Electrical And Computer Engineering     Masters     Accepted   
...                                      ...         ...          ...   
97589                         Bioengineering         PhD     Accepted   
99329   Mechanical And Aerospace Engineering     Masters     Accepted   
99344                     Genetic Counseling     Masters     Accepted   
100027                            Accounting         PhD     Rejected   
100028                            Accounting         PhD     Rejected   

        undergrad_gpa  gre_quantitative_reasoning  gre_verbal_reasoning  \
679              3.90                         169                     0   
3357             3.02                         140                     0   
3628             4.00                         139                     0   
5955             2.20                         138                     0   
8164             3.80                         136                     0   
...               ...                         ...                   ...   
97589            3.50                         165                     0   
99329            3.65                         161                     0   
99344            3.63                         165                     0   
100027           3.97                         169                     0   
100028           3.97                         169                     0   

        analytical_writing                                              notes  \
679                    3.8                              email to check portal   
3357                   2.0                                                NaN   
3628                   5.9  After what I did to Rachel, the dean of admiss...   
5955                   2.6  The groundhog saw its shadow, predicting 6 mor...   
8164                   3.5  Received the status update email around 12:30 ...   
...                    ...                                                ...   
97589                  5.3                                                NaN   
99329                  5.0                                                NaN   
99344                  4.0                     Applied to 6

In [6367]:
fig = px.histogram(valid_gre_verbal_reasoning_profiles, x=list_of_gre_columns[1], nbins=100, range_x=[129,171],
                   title=f'Distribution of Valid {list_of_gre_columns[1]} Scores')
fig.show()

In [6368]:
# Count of missing
missing_count = df['gre_verbal_reasoning'].isna().sum()

# Step 1: Generate truncated normal values in 130–170
mean, std = 149, 8  # Approx based on visual and given stats
lower, upper = 130, 170
a, b = (lower - mean) / std, (upper - mean) / std
random_vals = truncnorm.rvs(a, b, loc=mean, scale=std, size=missing_count)

# Step 2: Adjust to match exact target mean
existing_vals = df['gre_verbal_reasoning'].dropna()
target_mean = 149
target_total = target_mean * (len(existing_vals) + missing_count)
required_sum = target_total - existing_vals.sum()

scaling_factor = required_sum / np.sum(random_vals)
adjusted_vals = np.clip(random_vals * scaling_factor, 130, 170)

# Step 3: Fill in the values
df.loc[df['gre_verbal_reasoning'].isna(), 'gre_verbal_reasoning'] = adjusted_vals.round()


/var/folders/dw/fn12lc3x2b317ydc5b70v2340000gn/T/ipykernel_73369/4224175989.py:16: RuntimeWarning:

divide by zero encountered in scalar divide



In [6369]:
(df[list_of_gre_columns[1]]).isna().sum()

0

In [6370]:
median_val = df[list_of_gre_columns[1]].median()
mode_val = df[list_of_gre_columns[1]].mode()[0]  # mode() returns a Series
mean_val = df[list_of_gre_columns[1]].mean()

print(f"Median: {median_val}, Mode: {mode_val}, Mean: {mean_val}")


Median: 149.0, Mode: 148, Mean: 148.9225242236025


In [6371]:
df

Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0      id  acceptance_rate  \
0                  0             0           0  879550             31.0   
1                  1             1           1  879549             32.0   
2                  2             2           2  879548             45.0   
3                  3             3           3  879547             44.0   
4                  4             4           4  879546             40.0   
...              ...           ...         ...     ...              ...   
100620        100620        100620      100620  739462             44.0   
100621        100621        100621      100621  739461             36.0   
100622        100622        100622      100622  739460             43.0   
100623        100623        100623      100623  739459             45.0   
100624        100624        100624      100624  739458             45.0   

                                             institution  \
0       Washington University In St. Louis (WashU/WUSTL)   
1                               University of California   
2                                      Purdue University   
3                                 University of Illinois   
4                                    Columbia University   
...                                                  ...   
100620                   Georgia Institute of Technology   
100621                           University of Rochester   
100622                             University of Florida   
100623                 University of Southern California   
100624                       California State University   

                                   program degree_type     decision  \
0                                  English         PhD     Rejected   
1       Bioinformatics and Systems Biology         PhD     Rejected   
2                                Sociology         PhD     Rejected   
3                                  Physics         PhD     Accepted   
4                                  History         PhD     Rejected   
...                                    ...         ...          ...   
100620                    Computer Science         PhD     Rejected   
100621                           Economics         PhD  Wait listed   
100622                           Economics         PhD        Other   
100623                   Political Science         PhD     Rejected   
100624           Speech Language Pathology     Masters        Other   

        undergrad_gpa  gre_quantitative_reasoning  gre_verbal_reasoning  \
0            3.947261                         134                   136   
1            3.990000                         140                   148   
2            3.974965                         132                   143   
3            3.560000                         132                   149   
4            3.840593                         139                   135   
...               ...                         ...                   ...   
100620       3.826565                         166                   137   
100621       3.877921                         168                   156   
100622       3.668602                         167                   148   
100623       3.622368                         170                   158   
100624       3.662229                         170                   139   

        analytical_writing                                              notes  \
0                      4.6                                                NaN   
1                      2.5  Rejected at midnight PDT after interview. my C...   
2                      4.9                                                NaN   
3                      2.7  Acceptance letter from the department, officia...   
4                      2.7                                     email from POI   
...                    ...                                                ...   
100620                 3.0                                       

In [6372]:
df['analytical_writing'] = pd.to_numeric(df[list_of_gre_columns[2]], errors = 'coerce')

In [6373]:
gre_analytical_writing_min = df[list_of_gre_columns[2]] >= 0.0
gre_analytical_writing_max = df[list_of_gre_columns[2]] <= 6.0

In [6374]:
valid_analytical_profile = df[gre_analytical_writing_min & gre_analytical_writing_max]
len(valid_analytical_profile)

100625

In [6375]:
# df.to_excel("../data/processed/preprocessed_final.xlsx")

In [6376]:
invalid_cases_for_gre_analytical = (
    (df[list_of_gre_columns[2]] < 1.0) |
    (df[list_of_gre_columns[2]] > 6.0)  # This overlaps with < 130 and < 170
).sum()

print(f"Invalid rows for {list_of_gre_columns[2]}: {invalid_cases_for_gre_analytical}")

Invalid rows for analytical_writing: 0


In [6377]:
df[list_of_gre_columns[2]] = pd.to_numeric(df[list_of_gre_columns[2]], errors=  'coerce')

In [6378]:
invalid_rows = df[(df[list_of_gre_columns[2]] < 1.0) | (df[list_of_gre_columns[2]] > 6.0)]

print("Invalid rows count:", len(invalid_rows))
invalid_rows.head()

Invalid rows count: 0


Empty DataFrame
Columns: [Unnamed: 0.2, Unnamed: 0.1, Unnamed: 0, id, acceptance_rate, institution, program, degree_type, decision, undergrad_gpa, gre_quantitative_reasoning, gre_verbal_reasoning, analytical_writing, notes, gre_total, decision_encoded]
Index: []

In [6379]:
# Step 1: Work on a copy to protect the original DataFrame
temp_df = valid_analytical_profile.copy()

# Step 2: Clean and clip invalid scores
temp_df['analytical_writing_clipped'] = pd.to_numeric(
    temp_df['analytical_writing'], errors='coerce'
).apply(lambda x: x if 1.0 <= x <= 6.0 else np.nan)

# Step 3: Round to nearest 0.1
temp_df['analytical_writing_clipped'] = temp_df['analytical_writing_clipped'].round(1)

# Step 4: Plot histogram
fig = px.histogram(
    temp_df.dropna(subset=['analytical_writing_clipped']),
    x='analytical_writing_clipped',
    nbins=int((6.0 - 1.0) / 0.1),
    title='Distribution of GRE Analytical Writing Scores',
    labels={'analytical_writing_clipped': 'Analytical Writing Score'}
)

# Step 5: Format x-axis
fig.update_xaxes(
    tickmode='linear',
    tick0=1.0,
    dtick=0.3,
    range=[1.0, 6.0]
)

fig.update_layout(
    bargap=0.05,
    xaxis_title='Analytical Writing Score',
    yaxis_title='Count',
    title_x=0.5
)

fig.show()


In [6380]:
df.loc[(df[list_of_gre_columns[2]] < 1.0) | (df[list_of_gre_columns[2]] > 6.0), list_of_gre_columns[2]] = 0

In [6381]:
# Step 1: Ensure numeric type (if not already done)
df[list_of_gre_columns[2]] = pd.to_numeric(df[list_of_gre_columns[2]], errors='coerce')

# Step 2: Identify rows where value is 0 (i.e., needs imputation)
zero_indices = df[df[list_of_gre_columns[2]] == 0].index
zero_count = len(zero_indices)

# Step 3: Define proportions
high_pct = 0.60
mid_pct = 0.30
low_pct = 0.10

high_count = int(high_pct * zero_count)
mid_count = int(mid_pct * zero_count)
low_count = zero_count - high_count - mid_count  # Ensure total matches

# Step 4: Generate imputed values per range
high_vals = [round(random.uniform(4.0, 6.0), 1) for _ in range(high_count)]
mid_vals = [round(random.uniform(2.5, 4.0), 1) for _ in range(mid_count)]
low_vals = [round(random.uniform(1.0, 2.5), 1) for _ in range(low_count)]

# Step 5: Combine and shuffle
generated_vals = high_vals + mid_vals + low_vals
random.shuffle(generated_vals)

# Step 6: Fill the 0s with generated values
df.loc[zero_indices, list_of_gre_columns[2]] = generated_vals


In [6382]:
final_values = (df[list_of_gre_columns[2]] < 1.0) | (df[list_of_gre_columns[2]] > 6.0) 
final_values.sum()

0

In [6383]:
# df.to_excel("../data/processed/preprocessed_final.xlsx")